In [0]:
project_name = 'test_project_1'
permissions='default'
#permissions='fulldata'

In [0]:
pip install pycountry

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%run /Snippets/header_008

PROJECT DETAILS:
Project identifier: test_project_1
Cluster data access level: default


EXECUTION DETAILS:
Spark version: 11.3.x-scala2.12, 
Cluster Node Type: i3.xlarge, 
Driver Node Type: c4.2xlarge, 
Worker Node Type: i3.xlarge


In [0]:
import pandas as pd
import pyspark.sql.functions as F
import pyspark.sql.column as C

DATA SNAPSHOT DETAILS:
This notebook uses the latest snapshot date for this version: "v008.20230509104116/", (09 May 2023)

 The following are the Scopus snapshot dates available for datasets at v008:
['v008.20220314092239/', 'v008.20220906063204/', 'v008.20221021151538/', 'v008.20230509104116/']
(If you want to use a different snapshot, in the first cell of your notebook, set the variable `custom_snapshot_date` to one of the options listed above (string data type))



DATA SAMPLE DETAILS:
Setting paths to run on sample datasets (~1% data volume sample with 1-hop citation links and associated profiles)


In [0]:
import pandas as pd
import pyspark.sql.functions as F
def first(lst):
  return lst[0]
def extract_publication_info(newdf):
  #newdf = df_ani.limit(50)
  newdf = newdf.select('doi','Eid','year','Au','Au_cors','publication_type','openaccess','issn','source','news_cnt','correspondence')


  newdf = newdf.withColumn('Au1', first(newdf['Au']))



  newdf = newdf.withColumn('Au2', F.element_at(newdf['Au'], -1))

  newdf = newdf.withColumn('num_au', F.size(newdf['Au']))

  newdf = newdf.withColumn('Au1_id', F.col('Au1').getItem('auid'))

  newdf = newdf.withColumn('Au2_id', F.col('Au2').getItem('auid'))

  newdf = newdf.withColumn('Au_cor_id', F.col('Au_cors').getItem('auid'))
  newdf = newdf.withColumn('Au_cor_name', F.col('Au_cors').getItem('given_name'))
  newdf = newdf.withColumn('cor1_given_name', first(newdf['Au_cor_name']))
  return newdf
def get_smc_classification(newdf):
  df_smc_ani = spark.read.format("parquet").load(basePath+tablename_smc_complete)
  newdf2 = newdf.join(df_smc_ani,'Eid', 'inner')
  return newdf2

def get_gender(newdf):
  df_gender = spark.read.format("parquet").load(basePath+tablename_gender_inference)
  #non_null_count = newdf.where(F.col('Au1_id').isNotNull()).count()
  mergedf = df_gender.join(newdf, newdf.Au1_id == df_gender.auid, 'inner').select(
      df_gender.Inferred_Probable_Gender,newdf.doi,newdf.news_cnt,newdf.Au,newdf.Eid, newdf.year, newdf.publication_type,newdf.openaccess, newdf.issn, 
      newdf.num_au, newdf.Au1_id,newdf.Au2_id,newdf.Au_cor_id, newdf.srcid, newdf.subfield_article, newdf.subfield_journal, newdf.subfield_hybrid,newdf.correspondence,
      newdf.cor1_given_name)
  mergedf = mergedf.withColumnRenamed('Inferred_Probable_Gender', 'Au1_gender')

  mergedf2 = df_gender.join(mergedf, mergedf.Au2_id == df_gender.auid, 'inner').select(
      df_gender.Inferred_Probable_Gender, mergedf.doi,mergedf.news_cnt,mergedf.Au,mergedf.Eid, mergedf.year, mergedf.publication_type,mergedf.openaccess, mergedf.issn, 
      mergedf.num_au, mergedf.Au1_id,mergedf.Au2_id,mergedf.Au_cor_id, mergedf.srcid, mergedf.subfield_article, mergedf.subfield_journal, mergedf.subfield_hybrid, mergedf.Au1_gender,mergedf.correspondence,mergedf.cor1_given_name)
  mergedf2 = mergedf2.withColumnRenamed('Inferred_Probable_Gender', 'Au2_gender')

  #mergedf2 = mergedf2.withColumn('no_cor', F.size(mergedf2['Au_cor_id']))
  #display("len os final4 ", mergedf2.count(),"\n")

  mergedf2 = mergedf2.withColumn('cor_au1', first(mergedf2['Au_cor_id']))

  #display(final_data4.limit(20))

  mergedf3 = mergedf2.filter(F.col('cor_au1').isNotNull())
  display("no of not null cor au",mergedf3.count(),"\n")

  mergedf4 = df_gender.join(mergedf3, mergedf3.cor_au1 == df_gender.auid, 'inner').select(
      df_gender.Inferred_Probable_Gender, mergedf3.doi,mergedf3.news_cnt,mergedf3.Au,mergedf3.Eid, mergedf3.year, mergedf3.publication_type,mergedf3.openaccess, mergedf3.issn, 
      mergedf3.num_au, mergedf3.Au1_id,mergedf3.Au2_id,mergedf3.Au_cor_id, mergedf3.srcid, mergedf3.subfield_article, mergedf3.subfield_journal, mergedf3.subfield_hybrid, mergedf3.Au1_gender,mergedf3.Au2_gender,mergedf3.cor_au1,mergedf3.correspondence,mergedf3.cor1_given_name)
  mergedf4 = mergedf4.withColumnRenamed('Inferred_Probable_Gender', 'cor_gender')

  return mergedf4

def merge_news_cnt(scop):
  scop = scop.filter(func.col('year') >= 2018)
  scop = scop.filter(func.col('year') <= 2022)
  df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/arabi2@wisc.edu/doi_news_final.csv")
  display("# doi data from crossref and altmetric ",df1.count(),"\n")
  df = scop.join(df1, 'doi','inner')
  df = df.withColumn("news_cnt", df["news_cnt"].cast(IntegerType()))
  return df

from pyspark.sql.functions import lower

df_ani = spark.read.format("parquet").load(basePath+tablename_ani)

display("len of df_ani ",df_ani.count(),"\n")
dd = df_ani.withColumn('language', first(df_ani['meta_language']))
d1 = dd.withColumn('language_s', lower(dd['language']))

d2 = d1.filter(func.col('language_s') == 'english')
display("len of df_ani in english ",d2.count(),"\n")

with_news = merge_news_cnt(d2)
display("merged data len between crossref and scopus ",with_news.count(),"\n")

newdf = extract_publication_info(with_news)

df_class = get_smc_classification(newdf)
final_data = get_gender(df_class)
display(final_data.count(),"\n")
final_data2 = final_data.dropna(subset=["doi"])
display(final_data2.count(),"\n")

##extracting sci_val topic for each publication
df_top_eids = spark.read.format("parquet").load(basePath+tablename_top_topic_eid)

final_data3 = final_data2.join(df_top_eids,'Eid','inner')
#display(final_data3.count())
display(final_data3.limit(10))

'len of df_ani '864655'\n''len of df_ani in english '844293'\n''# doi data from crossref and altmetric '9061001'\n''merged data len between crossref and scopus '113617'\n''no of not null cor au'106605'\n'106461'\n'106461'\n'

Eid,cor_gender,doi,news_cnt,Au,year,publication_type,openaccess,issn,num_au,Au1_id,Au2_id,Au_cor_id,srcid,subfield_article,subfield_journal,subfield_hybrid,Au1_gender,Au2_gender,cor_au1,correspondence,cor1_given_name,TopicId
85120608377,unknown,10.3390/s21238155,0,"List(List(1, 57363708500, auth, false, null, 7f79731916c869a85075bf299ee349e01fc32ea4, Nivesh, Nivesh, Gadipudi N., Gadipudi N., N., N., null, null, null, Gadipudi, Gadipudi), List(2, 6503849022, auth, false, null, f2add25904ec270a5fae5aafe45473844853da6b, Irraivan, Irraivan, Elamvazuthi I., Elamvazuthi I., I., I., null, null, null, Elamvazuthi, Elamvazuthi), List(3, 55699384800, auth, false, null, 985445ea16167ad84665b03d8963e8508c6eaa80, Cheng-Kai, Cheng Kai, Lu C.-K., Lu C.K., C.-K., C.K., null, null, null, Lu, Lu), List(4, 55662378400, auth, false, null, dfecc2c32280d8a973d52396e251d3c63658330e, Sivajothi, Sivajothi, Paramasivam S., Paramasivam S., S., S., null, null, null, Paramasivam, Paramasivam), List(5, 57216312578, auth, false, null, 32aa708b7b25e0593e6b162d05d9ea597c54f3c3, Steven, Steven, Su S., Su S., S., S., null, null, null, Su, Su))",2021,ar,Full,14248220,5,57363708500,57216312578,List(6503849022),130124,artificial intelligence & image processing,analytical chemistry,analytical chemistry,male,male,6503849022,"List(List(mys, null))",Irraivan,5576
85132752517,unknown,10.1007/s00521-022-07484-y,0,"List(List(1, 57363708500, auth, false, null, 7f79731916c869a85075bf299ee349e01fc32ea4, Nivesh, Nivesh, Gadipudi N., Gadipudi N., N., N., null, null, null, Gadipudi, Gadipudi), List(2, 6503849022, auth, false, null, f2add25904ec270a5fae5aafe45473844853da6b, Irraivan, Irraivan, Elamvazuthi I., Elamvazuthi I., I., I., null, 0000-0002-4721-9400, null, Elamvazuthi, Elamvazuthi), List(3, 55699384800, auth, false, null, 985445ea16167ad84665b03d8963e8508c6eaa80, Cheng-Kai, Cheng Kai, Lu C.-K., Lu C.K., C.-K., C.K., null, null, null, Lu, Lu), List(4, 55662378400, auth, false, null, dfecc2c32280d8a973d52396e251d3c63658330e, Sivajothi, Sivajothi, Paramasivam S., Paramasivam S., S., S., null, null, null, Paramasivam, Paramasivam), List(5, 7402029883, auth, false, null, 32aa708b7b25e0593e6b162d05d9ea597c54f3c3, Steven, Steven, Su S., Su S., S., S., null, null, null, Su, Su))",2022,ar,None,09410643,5,57363708500,7402029883,List(6503849022),24800,artificial intelligence & image processing,artificial intelligence & image processing,artificial intelligence & image processing,male,unknown,6503849022,"List(List(mys, null))",Irraivan,5576
85142297568,unknown,10.1007/s00371-022-02702-z,0,"List(List(1, 57363708500, auth, false, null, 7f79731916c869a85075bf299ee349e01fc32ea4, Nivesh, Nivesh, Gadipudi N., Gadipudi N., N., N., null, null, null, Gadipudi, Gadipudi), List(2, 6503849022, auth, false, null, f2add25904ec270a5fae5aafe45473844853da6b, Irraivan, Irraivan, Elamvazuthi I., Elamvazuthi I., I., I., null, 0000-0002-4721-9400, null, Elamvazuthi, Elamvazuthi), List(3, 23005041200, auth, false, null, e934c4a387844b3f8eb0297db007cf743cf7457c, Lila Iznita, Lila Iznita, Izhar L.I., Izhar L.I., L.I., L.I., null, null, null, Izhar, Izhar), List(4, 57191245197, auth, false, null, 36765f1a62383b6b89a1622b3147e3e8ad729162, Lokender, Lokender, Tiwari L., Tiwari L., L., L., null, null, null, Tiwari, Tiwari), List(5, 54383443000, auth, false, null, 63149efb383a65711b986c5da8f45532672f5b9a, Ramya, Ramya, Hebbalaguppe R., Hebbalaguppe R., R., R., null, null, null, Hebbalaguppe, Hebbalaguppe), List(6, 55699384800, auth, false, null, 0688e65dee545d4c0d59054e6db2100d85c073b4, Cheng-Kai, Cheng Kai, Lu C.-K., Lu C.K., C.-K., C.K., null, null, null, Lu, Lu), List(7, 57658030800, auth, false, null, 426134d346255137626b1dcd787898ffb34e6476, Arockia Selvakumar Arockia, Arockia Selvakumar Arockia, Doss A.S.A., Doss A.S.A., A.S.A., A.S.A., null, null, null, Doss, Doss))",2022,ar,None,01782789,7,57363708500,57658030800,List(6503849022),26146,industrial engineering & automation,software engineering,software e

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.column as C


def find_sjr(met,yy,yr):

  ylst = F.tolist(yy)

  #index = ylst.index(yr)
  #sjr = m[index]
  return m


df_sources = spark.read.format("parquet").load(basePath+tablename_sources)

data = final_data3.join(df_sources,'srcid','inner').select(df_sources.metrics_calculations,df_sources.sourcetitle,df_sources.openaccess_status,final_data3.srcid,final_data3.doi,final_data3.news_cnt,final_data3.year,
final_data3.publication_type, final_data3.issn, final_data3.num_au,final_data3.Au1_id, final_data3.subfield_journal, final_data3.Au1_gender,final_data3.TopicId, 
final_data3.cor_gender,final_data3.cor_au1,final_data3.correspondence, final_data3.cor1_given_name)

display("len of merged data between src ", data.count(),"\n")

data = data.withColumn('SJR_lst', F.col('metrics_calculations').getItem('SJR'))
data = data.withColumn('year_SJR', F.col('metrics_calculations').getItem('year'))

#data = data.withColumn('SJR', find_sjr(data['SJR_lst'],data['year_SJR'], data['year']))
data = data.withColumn("pos", F.expr("array_position(year_SJR, year)"))


data = data.withColumn("pos", data["pos"].cast(IntegerType()))  
data = data.withColumn("SJR", F.element_at(data['SJR_lst'], data['pos']))

data = data.withColumn('cor_af', first(data['correspondence']))
data = data.withColumn('cor_coun', F.col('cor_af').getItem('country'))



import pycountry
import pyspark.sql.functions as F

def convert_country_short_name(country_short_name):
  #country = pycountry.countries.get(alpha_2=country_short_name)
  if len(country_short_name) == 3:
    country = pycountry.countries.get(alpha_3=country_short_name)
  else:
    country = pycountry.countries.get(alpha_2=country_short_name)

  return country.name if country else None

@F.udf(returnType=F.StringType())
def my_udf(country_short_name):
  return convert_country_short_name(country_short_name)

data = data.filter(F.col('cor_coun').isNotNull())
data = data.withColumn("country_name", my_udf(data["cor_coun"]))
#un_full_coun = un_name.withColumn("country_name", my_udf(un_name["cor_coun"]))



val_data = data.filter(func.col('cor_gender') != 'unknown')

#val_un = data.filter(func.col('cor_gender') == 'unknown')



display("len of data whose cor_gender is not unknown ",val_data.count(),"\n")

display(val_data.limit(5),"\n")

'len of merged data between src '106414'\n''len of data whose cor_gender is not unknown '63943'\n'

metrics_calculations sourcetitle openaccess_status srcid doi news_cnt year publication_type issn num_au Au1_id subfield_journal Au1_gender TopicId cor_gender cor_au1 correspondence cor1_given_name SJR_lst year_SJR pos SJR cor_af cor_coun country_name List(List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 1996, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 1997, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 1998, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 1999, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 2000, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 2001, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 2002, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 2003, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 2004, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 2005, 0, 0), List(null, null, 0.0, 0.0, null, 0, 0.0, Complete, 2006, 0, 0), List(null, null, 7.0, 7.0, null, 1, 0.0, Complete, 2007, 0, 0), List(null, null, 5.0, 5.0, null, 0, 0.0, Complete, 2008, 0, 0), List(null, null, 10.0, 6.0, null, 74, 5.41, Complete, 2009, 2, 2), List(0.333, 0.718, 158.0, 117.0, null, 144, 5.56, Complete, 2010, 1, 1), List(0.533, 1.351, 416.0, 359.0, List(List(334, 1.1, 46, 295, List(List(1900, List(154, 62), 60)))), 140, 4.29, Complete, 2011, 2, 2), List(0.99, 1.759, 1192.0, 840.0, List(List(1426, 2.7, 74, 522, List(List(1900, List(162, 22), 86)))), 186, 1.08, Complete, 2012, 0, 2), List(1.127, 1.96, 2376.0, 1635.0, List(List(2872, 3.7, 70, 781, List(List(1900, List(167, 14), 91)))), 362, 1.66, Complete, 2013, 2, 4), List(1.275, 1.856, 3852.0, 2957.0, List(List(4417, 3.9, 67, 1128, List(List(1900, List(170, 17), 90)))), 541, 2.4, Complete, 2014, 3, 6), List(1.349, 1.691, 6816.0, 5235.0, List(List(7153, 4.3, 68, 1671, List(List(1900, List(171, 17), 90)))), 778, 1.29, Complete, 2015, 11, 17), List(1.309, 1.737, 10734.0, 8565.0, List(List(12359, 5.2, 78, 2389, List(List(1900, List(174, 17), 90)))), 1032, 1.26, Complete, 2016, 7, 10), List(1.386, 1.642, 16744.0, 13315.0, List(List(18595, 5.3, 74, 3529, List(List(1900, List(183, 15), 92)))), 1322, 0.98, Complete, 2017, 13, 20), List(1.43, 1.93, 27112.0, 21110.0, List(List(28083, 5.6, 74, 4977, List(List(1900, List(184, 16), 91)))), 2072, 1.74, Complete, 2018, 30, 43), List(1.422, 1.813, 43460.0, 32478.0, List(List(43255, 6.1, 75, 7040, List(List(1900, List(187, 15), 92)))), 3051, 1.54, Complete, 2019, 30, 61), List(1.285, 1.705, 64197.0, 47199.0, List(List(65645, 6.6, 76, 9965, List(List(1900, List(186, 18), 90)))), 4193, 2.1, Complete, 2020, 94, 173), List(1.283, 1.546, 91210.0, 69962.0, List(List(102139, 7.4, 77, 13890, List(List(1900, List(191, 17), 91)))), 5146, 1.87, Complete, 2021, 341, 657), List(null, null, 125221.0, 98706.0, List(List(143194, 7.9, 79, 18092, List(List(1900, List(196, 18), 91)))), 6346, 1.89, In-Progress, 2022, 2299, 3207), List(null, null, 34205.0, 27897.0, null, 1459, 2.19, In-Progress, 2023, 1356, 1412)) Remote Sensing YES 86430 10.3390/rs11202418 0 2019 ar 20724292 7 57210823248 geological & geomatics engineering male 395 male 57210823248 List(List(esp, null)) Pablo List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, 0.333, 0.533, 0.99, 1.127, 1.275, 1.349, 1.309, 1.386, 1.43, 1.422, 1.285, 1.283, null, null) List(1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023) 24 1.422 List(esp, null) esp Spain List(List(null, null, 1475.0, 1472.0, null, 44, 68.18, Complete, 1996, 2, 2), List(null, null, 1545.0, 1544.0, null, 24, 0.0, Complete, 1997, 0, 0), List(null, null, 1521.0, 1519.0, null, 25, 0.0, Complete, 1998, 1, 1), List(3.871, 1.32, 1708.0, 1696.0, null, 48, 0.0, Complete, 1999, 2, 2), List(5.09, 1.386, 2030.0, 2023.0, null, 47, 0.0, Complete, 2000, 1, 1), List(2.998, 1.352, 2138.0, 2113.0, null, 52, 0.0, Comple

In [0]:
#val_un = data.filter(func.col('cor_gender') == 'unknown')
df_apr = spark.read.format("parquet").load(basePath+tablename_apr)
#un_full_coun = val_un.join(df_apr,val_un.cor_au1 == df_apr.auid,'inner')
un_full_coun = data.filter(func.col('cor_gender') == 'unknown')

display("len of unknown name ", un_full_coun.count(),"\n")
df_wos = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/arabi2@wisc.edu/wos_gender_firstname.csv")
#df_wos.printSchema()
#df_wos = df_wos.filter(func.col('country') == 'China')
newfile = un_full_coun.join(df_wos,[un_full_coun.cor1_given_name == df_wos.givenname,un_full_coun.country_name == df_wos.country],'inner')
display("len or merged gender with wos ", newfile.count(),"\n")

'len of unknown name '42136'\n''len or merged gender with wos '27647'\n'

In [0]:
ddf2 = newfile.drop("cor_gender")
ddf3 = ddf2.withColumnRenamed("gender", "cor_gender")
#ddf3.printSchema()
#display(ddf3.select('cor_gender').distinct())
ddf4 = ddf3.select('srcid','openaccess_status','doi','news_cnt','year','issn','num_au','cor_gender','TopicId','Au1_gender','country_name','SJR','cor_au1').toPandas()
print(ddf4.head(5))

        srcid openaccess_status                              doi  news_cnt  \
0       23918                NO        10.1007/s11356-018-3496-7         0   
1  4000148205                NO        10.1038/s41589-018-0133-1         0   
2       14305                NO           10.1093/sysbio/syab077         1   
3       27049                NO  10.1016/j.gloplacha.2020.103407         0   
4       23350                NO               10.1039/c8cs00201k         0   

   year      issn  num_au cor_gender  TopicId Au1_gender country_name     SJR  \
0  2019  09441344       4          M    17716    unknown      Algeria   0.788   
1  2018  15524450       1          F    12864    unknown    Australia   7.485   
2  2022  10635157       7          F     8566    unknown      Belgium     NaN   
3  2021  09218181       4          M    31888    unknown      Belgium   1.371   
4  2018  03060012      12          M      319       male      Austria  16.116   

       cor_au1  
0  55246627600  
1  2339279

In [0]:
ddfx = val_data.select('srcid','openaccess_status','doi','news_cnt','year','issn','num_au','cor_gender','TopicId','Au1_gender','country_name','SJR','cor_au1').toPandas()
display(len(ddfx),"\n")
display(len(ddf4),"\n")
ddf = ddfx.append(ddf4, ignore_index = True)
display(len(ddf),"\n")

63943'\n'27647'\n'91590'\n'

In [0]:
smc = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/arabi2@wisc.edu/sm_journal_classification.csv")
smc = smc.select('srcid','Field_English').toPandas()
smc = smc.dropna()
smc['srcid'] = smc['srcid'].astype(int)

#print(smc.dtypes)
total = smc.merge(ddf,on=['srcid'],how = 'inner')
print(len(total))

91588


In [0]:
df_ani = spark.read.format("parquet").load(basePath+tablename_ani)
a = df_ani.select('doi','Eid','year','Au','Au_cors')
a = a.withColumn('Au_id', F.col('Au').getItem('auid'))
a = a.select('year','Au_id').toPandas()
a = a.dropna()

def find(target_value):
  new = a[a['Au_id'].apply(lambda lst: target_value in lst)]
  #print("printing ")
  #print(new)
  return new['year'].min()

print("lenght of total ",len(total))
total = total.dropna(subset=['cor_au1'])
print("lenght of total after dropping null cor_au1 ",len(total))

total['min_yr'] = total['cor_au1'].apply(find)
total['career_length'] = total['year'] - total['min_yr']

lenght of total  91588
lenght of total after dropping null cor_au1  91588


# test only

In [0]:
df_ani = spark.read.format("parquet").load(basePath+tablename_ani)
newdf = df_ani.select('doi','Eid','year','Au','Au_cors')
newdf = newdf.withColumn('Au_id', F.col('Au').getItem('auid'))
a = newdf.select('year','Au_id','doi').toPandas()
a = a.dropna()

def find(target_value):
  new = a[a['Au_id'].apply(lambda lst: target_value in lst)]
  print("printing ")
  print(new)
  return new['year'].min()

datat = data.select('doi','year','cor_au1').toPandas()
datat = datat.dropna(subset=['cor_au1'])
datat = datat.head(2)
datat['min_yr'] = datat['cor_au1'].apply(find)
datat['career_length'] = datat['year'] - datat['min_yr']

printing 
        year                                              Au_id  \
139071  2018  [35106076300.0, 56333567900.0, 7801445497.0, 1...   
591371  2015  [35106076300.0, 36634914000.0, 56333567900.0, ...   
786862  2020  [35106076300.0, 57204466811.0, 55768108105.0, ...   

                              doi  
139071    10.1001/jama.2018.18020  
591371  10.1186/s12883-015-0294-4  
786862          10.1002/mus.26887  
printing 
        year                                              Au_id  \
248694  2018  [6701842148.0, 7404040049.0, 57196845704.0, 74...   
360573  2021  [56043547900.0, 7403882361.0, 36612565300.0, 5...   
409266  2018         [7402540730.0, 7004341520.0, 7404040049.0]   
553747  2018         [7402540730.0, 7004341520.0, 7404040049.0]   
599528  2017  [36612565300.0, 55295450300.0, 57194068726.0, ...   

                                      doi  
248694               10.1017/ice.2017.309  
360573  10.1001/jamanetworkopen.2021.0971  
409266             10.1001/jama.

In [0]:
print(datat)

                       doi  year      cor_au1  min_yr  career_length
0  10.1001/jama.2018.18020  2018  35106076300    2015              3
1   10.1001/jama.2018.6986  2018   7402540730    2017              1


In [0]:
from pyspark.sql.functions import array_contains
#from pyspark.sql.types import IntegerType
import pandas as pd
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as F
from pyspark.sql import types as t
from pyspark.sql import SparkSession 
from pyspark.sql.types import IntegerType


def career(aa,newdf):
  display(aa)
  result_df = newdf.filter(array_contains(newdf.Au_id, aa))
  
  display(result_df.limit(10))
  min_yr = result_df.agg({"year": "min"}).collect()[0][0]
  return min_yr

#combine_udf = udf(career, IntegerType())

'''@F.udf(returnType=t.IntegerType())
def combine_udf(aa,newdf):
  return career(aa,newdf)'''

combine_udf = udf(career, IntegerType())

df_ani = spark.read.format("parquet").load(basePath+tablename_ani)
newdf = df_ani.select('doi','Eid','year','Au','Au_cors')
  
newdf = newdf.withColumn('Au_id', F.col('Au').getItem('auid'))


da = data.limit(10)
da = da.withColumn('career_length', combine_udf(col('cor_au1'),newdf))
#da = da.withColumn('career_length', da['cor_au1'].apply(career, args=[da['year']]))

display(da)
#y = career(35378854900,2018)  

Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/serializers.py", line 527, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/databricks/spark/python/pyspark/context.py", line 495, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
/databricks/spark/python/pyspark/serializers.py in dumps(self, obj)
    526         try:
--> 527             return cloudpickle.dumps(obj, pickle_protocol)
    528         except pickle.PickleError:

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dumps(obj, protocol, buffer_callback)
     72             )
---> 73             cp.dump(obj)
     74             return file.getvalue()

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dump(self, obj)
    601         try:
--> 602             return Pickler.dump(self, obj)
    603         except RuntimeError as e:

/databricks/spark/python/pyspark/context.py in __getnewargs__(self)
    494         # This method is called when attempting to pickle SparkContext, which is always an error:
--> 495         raise RuntimeError(
    496             "It appears th

In [0]:
df_ani = spark.read.format("parquet").load(basePath+tablename_ani)
newdf = df_ani.select('doi','Eid','year','Au','Au_cors','issn','source','correspondence')
newdf = newdf.withColumn('Au_id', F.col('Au').getItem('auid'))
#display(newdf)
display(newdf.agg({"year": "max"}).collect()[0][0])
display(newdf.filter(func.col("year") == 2024))

2024

doi,Eid,year,Au,Au_cors,issn,source,correspondence,Au_id
10.1016/j.jes.2023.01.012,85150233802,2024,"List(List(1, 57461353000, auth, false, null, null, Cong, Cong, Lin C., Lin C., C., C., null, null, null, Lin, Lin), List(2, 57461353100, auth, false, null, null, Yuxuan, Yuxuan, Feng Y., Feng Y., Y., Y., null, null, null, Feng, Feng), List(3, 57210989010, auth, false, null, null, Xianjun, Xianjun, Xie X., Xie X., X., X., null, null, null, Xie, Xie), List(4, 57217077888, auth, false, null, null, Haoyu, Haoyu, Zhang H., Zhang H., H., H., null, null, null, Zhang, Zhang), List(5, 58128599900, auth, false, null, null, Jie, Jie, Wu J., Wu J., J., J., null, null, null, Wu, Wu), List(6, 58128519000, auth, false, null, null, Yixiao, Yixiao, Zhu Y., Zhu Y., Y., Y., null, null, null, Zhu, Zhu), List(7, 58128519100, auth, false, null, null, Jing, Jing, Yu J., Yu J., J., J., null, null, null, Yu, Yu), List(8, 57461781200, auth, false, null, null, Jingyi, Jingyi, Feng J., Feng J., J., J., null, null, null, Feng, Feng), List(9, 57460929000, auth, false, null, null, Wen, Wen, Su W., Su W., W., W., null, null, null, Su, Su), List(10, 58128558900, auth, false, null, null, Shanming, Shanming, Lai S., Lai S., S., S., null, null, null, Lai, Lai), List(11, 36122204600, auth, false, null, ee51cd6ad89f9b64ebe69a39ea0615ce5922bf92, Anyun, Anyun, Zhang A., Zhang A., A., A., null, 0000-0002-8814-517X, null, Zhang, Zhang))","List(List(11, 36122204600, null, false, null, null, Anyun, null, Zhang A., null, A., null, null, null, null, Zhang, null))",10010742,"List(null, null, chn, 01, 03, March 2024, 2024, null, null, null, null, null, List(18787320), null, List(10010742), Chinese Academy of Sciences, Journal of Environmental Sciences (China), J. Environ. Sci., 23393, j, http://www.journals.elsevier.com/journal-of-environmental-sciences/)","List(List(chn, null))","List(57461353000, 57461353100, 57210989010, 57217077888, 58128599900, 58128519000, 58128519100, 57461781200, 57460929000, 58128558900, 36122204600)"
10.1016/j.jes.2022.12.022,85147549544,2024,"List(List(1, 56681557400, auth, false, null, null, Yiruiwen, Yiruiwen, Xie Y., Xie Y., Y., Y., null, 0000-0002-5443-2699, null, Xie, Xie), List(2, 56291941000, auth, false, null, 98a0bb4ebde1484ab2bb69ae152d8dc414a97b47, Dao, Dao, Guan D., Guan D., D., D., null, 0000-0002-1479-9808, null, Guan, Guan), List(3, 57217448218, auth, false, null, null, Yangfan, Yangfan, Deng Y., Deng Y., Y., Y., null, null, null, Deng, Deng), List(4, 57221700172, auth, false, null, null, Yugo, Yugo, Sato Y., Sato Y., Y., Y., null, 0000-0002-3896-0176, null, Sato, Sato), List(5, 57218451051, auth, false, null, null, Yu, Yu, Luo Y., Luo Y., Y., Y., null, null, null, Luo, Luo), List(6, 8082389600, auth, false, null, a5eb42cec72c85c0d1f2c4f2dfed2f0c168d8fbf, Guanghao, Guanghao, Chen G., Chen G., G., G., null, null, null, Chen, Chen))","List(List(2, 56291941000, null, false, null, null, Dao, null, Guan D., null, D., null, null, null, null, Guan, null))",10010742,"List(null, null, chn, 01, 01, January 2024, 2024, null, null, null, null, null, List(18787320), null, List(10010742), Chinese Academy of Sciences, Journal of Environmental Sciences (China), J. Environ. Sci., 23393, j, http://www.journals.elsevier.com/journal-of-environmental-sciences/)","List(List(chn, null))","List(56681557400, 56291941000, 57217448218, 57221700172, 57218451051, 8082389600)"
10.1016/j.jes.2023.02.023,85150362874,2024,"List(List(1, 56918861700, auth, false, null, null, Peng, Peng, Chen P., Chen P., P., P., null, null, null, Chen, Chen), List(2, 57206819946, auth, false, null, null, Yi, Yi, Liu Y., Liu Y., Y., Y., null, null, null, Liu, Liu), List(3, 55745721000, auth, false, null, 3191c5dd0b720b794cfb4ba6369add536ee1d33b, Guo-Xin, Guo Xin, Sun G.-X., Sun G.X., G.-X., G.X., null, null, null, Sun, Sun))","List(List(3, 55745721000, null, false, null, null, Guo-Xin, null, Sun G.-X., null, G.-X., null, null, null, null, Sun, null))",10010742,"List(null, null, chn, 0

In [0]:

from pyspark.sql.functions import array_contains
import pandas as pd
import pyspark.sql.functions as F
def first(lst):
  return lst[0]

newdf = newdf.withColumn('Au1', first(newdf['Au_id']))
result_df = newdf.filter(array_contains(newdf.Au_id, 7202831850))
display(result_df)

doi,Eid,year,Au,Au_cors,issn,source,correspondence,Au_id,Au1
10.1097/00005650-200209000-00007,36717780,2002,"List(List(1, 35378854900, null, false, null, a7d8fd447b21455456b14b85e3283d5a3140b2f0, Christopher M., Christopher M., Callahan C.M., Callahan C.M., C.M., C.M., null, null, null, Callahan, Callahan), List(2, 35481291100, null, false, null, null, Frederick W., Frederick W., Unverzagt F.W., Unverzagt F.W., F.W., F.W., null, null, null, Unverzagt, Unverzagt), List(3, 7202831850, null, false, null, null, Siu L., Siu L., Hui S.L., Hui S.L., S.L., S.L., null, null, null, Hui, Hui), List(4, 7102761843, null, false, null, null, Anthony J., Anthony J., Perkins A.J., Perkins A.J., A.J., A.J., null, null, null, Perkins, Perkins), List(5, 7004923246, null, false, null, null, Hugh C., Hugh C., Hendrie H.C., Hendrie H.C., H.C., H.C., null, null, null, Hendrie, Hendrie))","List(List(1, 35378854900, null, false, Dr., null, null, null, Callahan C.M., null, C.M., null, null, null, null, Callahan, null))",00257079,"List(null, null, usa, null, 09, September 2002, 2002, null, null, null, null, null, null, null, List(00257079), null, Medical Care, Med. Care, 17785, j, null)","List(List(usa, null))","List(35378854900, 35481291100, 7202831850, 7102761843, 7004923246)",35378854900
10.1016/S0091-7435(02)00038-5,37325732,2003,"List(List(1, 7102774879, auth, false, null, ca37e0c63d09e4c99a9dba701f972d44ad58b41f, Victoria, Victoria, Champion V., Champion V., V., V., null, null, null, Champion, Champion), List(2, 6505991289, auth, false, null, null, Maltie, Maltie, Maraj M., Maraj M., M., M., null, null, null, Maraj, Maraj), List(3, 7202831850, auth, false, null, null, Siu, Siu, Hui S., Hui S., S., S., null, null, null, Hui, Hui), List(4, 7102761843, auth, false, null, null, Anthony J., Anthony J., Perkins A.J., Perkins A.J., A.J., A.J., null, null, null, Perkins, Perkins), List(5, 7103277128, auth, false, null, null, William, William, Tierney W., Tierney W., W., W., null, null, null, Tierney, Tierney), List(6, 36087734300, auth, false, null, null, Usha, Usha, Menon U., Menon U., U., U., null, null, null, Menon, Menon), List(7, 7201765053, auth, false, null, null, Celette Sugg, Celette Sugg, Skinner C.S., Skinner C.S., C.S., C.S., null, null, null, Skinner, Skinner))","List(List(1, 7102774879, null, false, null, null, null, null, Champion V., null, V., null, null, null, null, Champion, null))",00917435,"List(null, null, usa, 01, 02, 1 February 2003, 2003, null, null, null, null, null, null, null, List(00917435), Academic Press Inc., Preventive Medicine, Prev. Med., 17665, j, http://www.elsevier.com/inca/publications/store/6/2/2/9/3/4/index.htt)","List(List(usa, null))","List(7102774879, 6505991289, 7202831850, 7102761843, 7103277128, 36087734300, 7201765053)",7102774879
10.1172/JCI113523,23902418,1988,"List(List(1, 7202831850, auth, false, null, null, null, S. L., Hui S.L., Hui S.L., S.L., S.L., null, null, null, Hui, Hui), List(2, 7005042897, auth, false, null, null, null, C. W., Slemenda C.W., Slemenda C.W., C.W., C.W., null, null, null, Slemenda, Slemenda), List(3, 57646572200, auth, false, null, null, null, C. C., Johnston Jr. C.C., Johnston C.C., C.C., C.C., null, null, null, Johnston Jr., Johnston))",List(),00219738,"List(null, null, usa, null, null, 1988, 1988, null, null, null, null, null, null, null, List(00219738), null, Journal of Clinical Investigation, J. CLIN. INVEST., 15870, j, null)",List(),"List(7202831850, 7005042897, 57646572200)",7202831850


In [0]:
df = newdf.select('doi','Eid','year','issn','Au_id').toPandas()


In [0]:
def find(target_value):

  a = pd.DataFrame()
  a['id'] = [1,2,3,4]
  a['lst'] = [[1,3,4],[1,3,5],[5,6,2],[5,6,1]]

  new = a[a['lst'].apply(lambda lst: target_value in lst)]
  return len(new)

b = pd.DataFrame()
b['name'] = [1,2,3]

b['new'] = b['name'].apply(find)



#target_value = 1
#result_df = b[b['lst'].apply(lambda lst: target_value in lst)]


print(b)

   name  new
0     1    3
1     2    1
2     3    2


# affiliation find testing

In [0]:
import pandas as pd
import pyspark.sql.functions as F
def first(lst):
  return lst[0]
def extract_publication_info(newdf):
  #newdf = df_ani.limit(50)
  newdf = newdf.select('doi','Eid','year','Au','Au_Af','Au_cors','publication_type','openaccess','issn','source','news_cnt','correspondence', 'Af')


  newdf = newdf.withColumn('Au1', first(newdf['Au']))



  newdf = newdf.withColumn('Au2', F.element_at(newdf['Au'], -1))

  newdf = newdf.withColumn('num_au', F.size(newdf['Au']))

  newdf = newdf.withColumn('Au1_id', F.col('Au1').getItem('auid'))

  newdf = newdf.withColumn('Au2_id', F.col('Au2').getItem('auid'))

  newdf = newdf.withColumn('Au_cor_id', F.col('Au_cors').getItem('auid'))
  newdf = newdf.withColumn('Au_cor_name', F.col('Au_cors').getItem('given_name'))
  newdf = newdf.withColumn('cor1_given_name', first(newdf['Au_cor_name']))

  newdf = newdf.withColumn('cor_seq', F.col('Au_cors').getItem('Authorseq'))
  newdf = newdf.withColumn('cor_seq_first', first(newdf['cor_seq']))

  newdf = newdf.withColumn('auth_seq', F.col('Au_Af').getItem('Authorseq'))
  newdf = newdf.withColumn('aff_seq', F.col('Au_Af').getItem('affiliation_seq'))
  newdf = newdf.withColumn("auth_pos", F.expr("array_position(auth_seq, cor_seq_first)"))

  newdf = newdf.withColumn("auth_pos", newdf["auth_pos"].cast(IntegerType()))  
  #newdf = newdf.filter(F.col('aff_seq').isNotNull())
  newdf = newdf.filter(F.col('auth_pos') != 0)
  newdf = newdf.withColumn("aff", F.element_at(newdf['aff_seq'], newdf['auth_pos']))
  newdf = newdf.withColumn("aff_val", F.element_at(newdf['Af'], newdf['aff']))
  newdf = newdf.withColumn("cor_afid", F.col('aff_val').getItem('afid'))



  return newdf
def get_smc_classification(newdf):
  df_smc_ani = spark.read.format("parquet").load(basePath+tablename_smc_complete)
  newdf2 = newdf.join(df_smc_ani,'Eid', 'inner')
  return newdf2

def get_gender(newdf):
  df_gender = spark.read.format("parquet").load(basePath+tablename_gender_inference)
  #non_null_count = newdf.where(F.col('Au1_id').isNotNull()).count()
  mergedf = df_gender.join(newdf, newdf.Au1_id == df_gender.auid, 'inner').select(
      df_gender.Inferred_Probable_Gender,newdf.doi,newdf.news_cnt,newdf.Au,newdf.Eid, newdf.year, newdf.publication_type,newdf.openaccess, newdf.issn, 
      newdf.num_au, newdf.Au1_id,newdf.Au2_id,newdf.Au_cor_id, newdf.srcid, newdf.subfield_article, newdf.subfield_journal, newdf.subfield_hybrid,newdf.correspondence,
      newdf.cor1_given_name)
  mergedf = mergedf.withColumnRenamed('Inferred_Probable_Gender', 'Au1_gender')

  mergedf2 = df_gender.join(mergedf, mergedf.Au2_id == df_gender.auid, 'inner').select(
      df_gender.Inferred_Probable_Gender, mergedf.doi,mergedf.news_cnt,mergedf.Au,mergedf.Eid, mergedf.year, mergedf.publication_type,mergedf.openaccess, mergedf.issn, 
      mergedf.num_au, mergedf.Au1_id,mergedf.Au2_id,mergedf.Au_cor_id, mergedf.srcid, mergedf.subfield_article, mergedf.subfield_journal, mergedf.subfield_hybrid, mergedf.Au1_gender,mergedf.correspondence,mergedf.cor1_given_name)
  mergedf2 = mergedf2.withColumnRenamed('Inferred_Probable_Gender', 'Au2_gender')

  #mergedf2 = mergedf2.withColumn('no_cor', F.size(mergedf2['Au_cor_id']))
  #display("len os final4 ", mergedf2.count(),"\n")

  mergedf2 = mergedf2.withColumn('cor_au1', first(mergedf2['Au_cor_id']))

  #display(final_data4.limit(20))

  mergedf3 = mergedf2.filter(F.col('cor_au1').isNotNull())
  display("no of not null cor au",mergedf3.count(),"\n")

  mergedf4 = df_gender.join(mergedf3, mergedf3.cor_au1 == df_gender.auid, 'inner').select(
      df_gender.Inferred_Probable_Gender, mergedf3.doi,mergedf3.news_cnt,mergedf3.Au,mergedf3.Eid, mergedf3.year, mergedf3.publication_type,mergedf3.openaccess, mergedf3.issn, 
      mergedf3.num_au, mergedf3.Au1_id,mergedf3.Au2_id,mergedf3.Au_cor_id, mergedf3.srcid, mergedf3.subfield_article, mergedf3.subfield_journal, mergedf3.subfield_hybrid, mergedf3.Au1_gender,mergedf3.Au2_gender,mergedf3.cor_au1,mergedf3.correspondence,mergedf3.cor1_given_name)
  mergedf4 = mergedf4.withColumnRenamed('Inferred_Probable_Gender', 'cor_gender')

  return mergedf4

def merge_news_cnt(scop):
  scop = scop.filter(func.col('year') >= 2018)
  scop = scop.filter(func.col('year') <= 2022)
  df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/arabi2@wisc.edu/doi_news_final.csv")
  display("# doi data from crossref and altmetric ",df1.count(),"\n")
  df = scop.join(df1, 'doi','inner')
  df = df.withColumn("news_cnt", df["news_cnt"].cast(IntegerType()))
  return df

from pyspark.sql.functions import lower

df_ani = spark.read.format("parquet").load(basePath+tablename_ani)

display("len of df_ani ",df_ani.count(),"\n")
dd = df_ani.withColumn('language', first(df_ani['meta_language']))
d1 = dd.withColumn('language_s', lower(dd['language']))

d2 = d1.filter(func.col('language_s') == 'english')
display("len of df_ani in english ",d2.count(),"\n")

with_news = merge_news_cnt(d2)
display("merged data len between crossref and scopus ",with_news.count(),"\n")

newdf = extract_publication_info(with_news)
display(newdf.limit(20))

'''df_class = get_smc_classification(newdf)
final_data = get_gender(df_class)
display(final_data.count(),"\n")
final_data2 = final_data.dropna(subset=["doi"])
display(final_data2.count(),"\n")

##extracting sci_val topic for each publication
df_top_eids = spark.read.format("parquet").load(basePath+tablename_top_topic_eid)

final_data3 = final_data2.join(df_top_eids,'Eid','inner')
#display(final_data3.count())
display(final_data3.limit(10))'''

'len of df_ani '864655'\n''len of df_ani in english '844293'\n''# doi data from crossref and altmetric '9061001'\n''merged data len between crossref and scopus '113617'\n'

doi,Eid,year,Au,Au_Af,Au_cors,publication_type,openaccess,issn,source,news_cnt,correspondence,Af,Au1,Au2,num_au,Au1_id,Au2_id,Au_cor_id,Au_cor_name,cor1_given_name,cor_seq,cor_seq_first,auth_seq,aff_seq,auth_pos,aff,aff_val,cor_afid
10.1001/jama.2019.8859,85069306734,2019,"List(List(1, 55759215500, auth, false, MD, null, Fraukje A., Fraukje A., Ponds F.A., Ponds F.A., F.A., F.A., null, null, null, Ponds, Ponds), List(2, 55066502300, auth, false, MD,PhD, null, Paul, Paul, Fockens P., Fockens P., P., P., null, null, null, Fockens, Fockens), List(3, 12786657800, auth, false, BSc, null, Aaltje, Aaltje, Lei A., Lei A., A., A., null, null, null, Lei, Lei), List(4, 7101811971, auth, false, MD,PhD, null, Horst, Horst, Neuhaus H., Neuhaus H., H., H., null, null, null, Neuhaus, Neuhaus), List(5, 24175533300, auth, false, MD, null, Torsten, Torsten, Beyna T., Beyna T., T., T., null, null, null, Beyna, Beyna), List(6, 56568042800, auth, false, MD, null, Jennis, Jennis, Kandler J., Kandler J., J., J., null, null, null, Kandler, Kandler), List(7, 56253543400, auth, false, MD,PhD, null, Thomas, Thomas, Frieling T., Frieling T., T., T., null, null, null, Frieling, Frieling), List(8, 7103182534, auth, false, MD,PhD, null, Philip W. Y., Philip W.Y., Chiu P.W.Y., Chiu P.W.Y., P.W.Y., P.W.Y., null, null, null, Chiu, Chiu), List(9, 7409253910, auth, false, MD,PhD, null, Justin C. Y., Justin C.Y., Wu J.C.Y., Wu J.C.Y., J.C.Y., J.C.Y., null, null, null, Wu, Wu), List(10, 26632569200, auth, false, BSc, null, Vivien W. Y., Vivien W.Y., Wong V.W.Y., Wong V.W.Y., V.W.Y., V.W.Y., null, null, null, Wong, Wong), List(11, 7005323747, auth, false, MD,PhD, null, Guido, Guido, Costamagna G., Costamagna G., G., G., null, null, null, Costamagna, Costamagna), List(12, 6506652660, auth, false, MD, null, Pietro, Pietro, Familiari P., Familiari P., P., P., null, null, null, Familiari, Familiari), List(13, 7102556261, auth, false, MD,PhD, null, Peter J., Peter J., Kahrilas P.J., Kahrilas P.J., P.J., P.J., null, null, null, Kahrilas, Kahrilas), List(14, 6603899097, auth, false, MD,PhD, null, John E., John E., Pandolfino J.E., Pandolfino J.E., J.E., J.E., null, null, null, Pandolfino, Pandolfino), List(15, 7101876282, auth, false, MD,PhD, null, André J. P. M., André J.P.M., Smout A.J.P.M., Smout A.J.P.M., A.J.P.M., A.J.P.M., null, null, null, Smout, Smout), List(16, 6602466064, auth, false, MD,PhD, 3ab21a990ba4c39c61be7abae6c46c4120d77103, Albert J., Albert J., Bredenoord A.J., Bredenoord A.J., A.J., A.J., null, null, null, Bredenoord, Bredenoord))","List(List(1, 1, true), List(2, 1, true), List(3, 1, true), List(15, 1, true), List(16, 1, true), List(4, 2, true), List(5, 2, true), List(6, 2, true), List(7, 3, true), List(8, 4, true), List(10, 4, true), List(9, 5, true), List(11, 6, true), List(12, 6, true), List(11, 7, true), List(13, 8, true), List(14, 8, true))","List(List(16, 6602466064, null, false, MD,PhD, null, Albert J., null, Bredenoord A.J., null, A.J., null, null, null, null, Bredenoord, null))",ar,Full,00987484,"List(null, null, usa, 09, 07, 9 July 2019, 2019, smcleod@itsa.ucsf.edu, email, null, null, null, List(15383598), null, List(00987484), American Medical Association, JAMA - Journal of the American Medical Association, JAMA, 85291, j, http://jama.jamanetwork.com/journal.aspx)",7,"List(List(nld, null))","List(List(null, Amsterdam, null, Netherlands, List(60002483, 60001157), List(Department of Gastroenterology and Hepatology, Amsterdam UMC, University of Amsterdam), 3, null, null, nld, null, 60002483, 110835913), List(null, Düsseldorf, null, Germany, List(60015895), List(Department of Gastroenterology, Evangelisches Krankenhaus Düsseldorf), 2, null, null, deu, null, 60015895, 103823363), List(null, Düsseldorf, null, Germany, List(60016143), List(Department of Gastroenterology, Helios Klinikum Krefeld), 2, null, null, deu, null, 60016143, 118951395), List(null, Hong Kong, null, Hong Kong, List(60002798), List(Institute of Digestive Disease, Department of

Out[20]: 'df_class = get_smc_classification(newdf)\nfinal_data = get_gender(df_class)\ndisplay(final_data.count(),"\n")\nfinal_data2 = final_data.dropna(subset=["doi"])\ndisplay(final_data2.count(),"\n")\n\n##extracting sci_val topic for each publication\ndf_top_eids = spark.read.format("parquet").load(basePath+tablename_top_topic_eid)\n\nfinal_data3 = final_data2.join(df_top_eids,\'Eid\',\'inner\')\n#display(final_data3.count())\ndisplay(final_data3.limit(10))'

In [0]:
import pandas as pd
import pyspark.sql.functions as F
import pyspark.sql.column as C
df_ipr = spark.read.format("parquet").load(basePath+tablename_ipr)
a = df_ipr.filter(F.col("afid") == 60009343)
display(a)

afid,preferred_name,sort_name,address_part,postal_code,city,state,country,country_tag,name_variant,org_domain,org_type,parent,parent_preferred_name
60009343,Memorial Sloan-Kettering Cancer Center,Memorial Sloan-Kettering Cancer Center,1275 York Avenue,10021,New York,NY,United States,usa,"List(List(Memorial Sloan-kettering Cancer Center, 50961), List(Memorial Sloan Kettering Cancer Center, 42573), List(Mem. Sloan-kettering Cancer Center, 9608), List(Memorial Sloan-kettering Cancer Ctr., 1858), List(Sloan Kettering Institute, 1246), List(Memorial Sloan-kettering Can. Center, 1009), List(Mem. Sloan-kettering Cancer Cent., 996), List(Mem. Sloan Kettering Cancer Cent., 917), List(Mem. Sloan Kettering Cancer Center, 872), List(Sloan-kettering Institute, 659))",mskcc.org,hosp,null,null
